In [3]:
import librosa                          #for loading music and audio analysis
import os
import soundfile as sf                  #read and write sound files
import numpy as np
import matplotlib.pyplot as plt        #work like matlab
import matplotlib.style as ms
from tqdm import tqdm                  #progress 
import pickle                       #python object serialization

import IPython.display              #display python object in all front ends
import librosa.display              #
ms.use('seaborn-muted')             #seaborn-muted is style
import pandas as pd
%matplotlib inline 
import wave
import contextlib
import math
import random

In [4]:
columns = ['wav_file','start_time','end_time','label']
df_time = pd.DataFrame(columns=columns)
time_list = []

In [5]:
wav_file_path = '/home/rashmi/Dataset/single_file/wav/'
orig_wav_files = os.listdir(wav_file_path)
for orig_wav_file in tqdm(orig_wav_files):
        try:
            fname = wav_file_path + orig_wav_file;
            with contextlib.closing(wave.open(fname,'r')) as f:
                frames = f.getnframes()
                rate = f.getframerate()
                duration = frames / float(rate)
                time_list = [orig_wav_file,'0',duration,'']
                
                df_time = df_time.append(pd.DataFrame(time_list, index=columns).transpose(), ignore_index=True)
                
        except:
              print('An exception occured for {}'.format(orig_wav_file))

df_time.to_csv('/home/rashmi/Dataset/single_file/pre-processed-mfcc/time_slot.csv', index=False)

100%|██████████| 5/5 [00:00<00:00, 35.24it/s]


In [6]:
import librosa
import os
import soundfile as sf
import numpy as np
import math
import pickle

In [7]:
labels_df = pd.read_csv('/home/rashmi/Dataset/single_file/pre-processed-mfcc/time_slot.csv')
iemocap_dir = '/home/rashmi/Dataset/single_file/wav/'

In [ ]:
columns = ['wav_file','start_time','end_time','label']
df_time = pd.DataFrame(columns=columns)
time_list = []
wav_array = []

In [22]:
sr = 44100
audio_vectors = {}
wav_file_path = '/home/rashmi/Dataset/single_file/wav/'
orig_wav_files = os.listdir(wav_file_path)
for orig_wav_file in tqdm(orig_wav_files):
        try:
            orig_wav_vector, _sr = librosa.load(wav_file_path + orig_wav_file, sr=sr)
            wav_array.append(orig_wav_file)
            orig_wav_file, file_format = orig_wav_file.split('.')
            for index, row in labels_df[labels_df['wav_file'].str.contains(orig_wav_file)].iterrows():
                start_time, end_time, truncated_wav_file_name= row['start_time'], row['end_time'], row['wav_file']
                start_frame = math.floor(start_time * sr)
                end_frame = math.floor(end_time * sr)
                truncated_wav_vector = orig_wav_vector[start_frame:end_frame + 1]
                audio_vectors[truncated_wav_file_name] = truncated_wav_vector
        except:
            print('An exception occured for {}'.format(orig_wav_file))
with open('/home/rashmi/Dataset/single_file/pre-processed-mfcc/audio_vectors.pkl', 'wb') as f:
    pickle.dump(audio_vectors, f)

100%|██████████| 5/5 [00:02<00:00,  2.31it/s]


In [28]:
print(wav_array)
len_wav_array=len(wav_array)
labels_df = pd.read_csv('/home/rashmi/Dataset/single_file/pre-processed-mfcc/time_slot.csv')
iemocap_dir = '/home/rashmi/Dataset/single_file/wav/'

['Ses05M_script02_2_F034.wav', 'ses05m_impro06_M004.wav', 'Ses05f_impro08_F022.wav', 'Ses05F_impro08_F007.wav', 'Ses05F_impro01_F013.wav']


In [29]:
import pandas as pd

In [30]:
labels_path = '/home/rashmi/Dataset/single_file/pre-processed-mfcc/time_slot.csv'
audio_vectors_path = '/home/rashmi/Dataset/single_file/pre-processed-mfcc/audio_vectors.pkl'
labels_df = pd.read_csv(labels_path)

In [31]:
columns = ['wav_file', 'sig_mean', 'sig_std', 'rmse_mean', 'rmse_std', 'silence', 'harmonic', 'auto_corr_max', 'auto_corr_std','mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13']
df_features = pd.DataFrame(columns=columns)
hop_length = 512

In [32]:
audio_vectors = pickle.load(open('/home/rashmi/Dataset/single_file/pre-processed-mfcc/audio_vectors.pkl', 'rb'))
for index, row in tqdm(labels_df[labels_df['wav_file'].str.contains('')].iterrows()):
            try:
                wav_file_name = row['wav_file']
                y = audio_vectors[wav_file_name]

                feature_list = [wav_file_name]  # wav_file, label
                sig_mean = np.mean(abs(y))
                feature_list.append(sig_mean)  # sig_mean
                feature_list.append(np.std(y))  # sig_std

                rms = librosa.feature.rms(y + 0.0001)[0]
                feature_list.append(np.mean(rms))  # rms_mean
                feature_list.append(np.std(rms))  # rms_std

                silence = 0
                for e in rms:
                    if e <= 0.4 * np.mean(rms):
                        silence += 1
                silence /= float(len(rms))
                feature_list.append(silence)  # silence

                y_harmonic = librosa.effects.hpss(y)[0]
                feature_list.append(np.mean(y_harmonic) * 1000)  # harmonic (scaled by 1000)

                # based on the pitch detection algorithm mentioned here:
                # http://access.feld.cvut.cz/view.php?cisloclanku=2009060001
                cl = 0.45 * sig_mean
                center_clipped = []
                for s in y:
                    if s >= cl:
                        center_clipped.append(s - cl)
                    elif s <= -cl:
                        center_clipped.append(s + cl)
                    elif np.abs(s) < cl:
                        center_clipped.append(0)
                auto_corrs = librosa.core.autocorrelate(np.array(center_clipped))
                feature_list.append(1000 * np.max(auto_corrs)/len(auto_corrs))  # auto_corr_max (scaled by 1000)
                feature_list.append(np.std(auto_corrs))  # auto_corr_std

                #mfcc 
                mfccs=np.mean(librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13).T, axis=0)
                
                for i in range(0,13):
                    feature_list.append(mfccs[i])  # mfcc 
                    
                df_features = df_features.append(pd.DataFrame(feature_list, index=columns).transpose(), ignore_index=True)
            except Exception as e: print(e)

df_features.to_csv('/home/rashmi/Dataset/single_file/pre-processed-mfcc/audio_features.csv', index=False)

5it [00:23,  4.66s/it]


In [33]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
import matplotlib
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.utils.class_weight import compute_class_weight
import itertools
from IPython.display import display

%matplotlib inline

In [34]:
x_train = pd.read_csv('/home/rashmi/Dataset/pre-processed-mfcc/s2e/audio_train.csv')
x_test = pd.read_csv('/home/rashmi/Dataset/single_file/pre-processed-mfcc/audio_features.csv')

print(x_train.shape)
y_train = x_train['label']

del x_train['label']
del x_train['wav_file']
del x_test['wav_file']

print(x_train.shape, x_test.shape)

(1704, 23)
(1704, 21) (5, 21)


In [35]:
emotion_dict = {'ang': 0,
                'hap': 1,
                'sad': 2,
                'fea': 3,
                'sur': 4,
                'neu': 5}

emo_keys = list(['ang', 'hap', 'sad', 'fea', 'sur', 'neu'])

In [36]:
rf_classifier = RandomForestClassifier(n_estimators=1200, min_samples_split=25)
xgb_classifier = xgb.XGBClassifier(max_depth=7, learning_rate=0.008, objective='multi:softprob', 
                                   n_estimators=1200, sub_sample=0.8, num_class=len(emotion_dict),
                                   booster='gbtree', n_jobs=4)
mnb_classifier = MultinomialNB()

In [37]:
#eclf1 = VotingClassifier(estimators=[('rf', rf_classifier), ('xgb', xgb_classifier), ('mnb', mnb_classifier)], voting='soft')
eclf1 = xgb_classifier.fit(x_train, y_train)
pred_probs=eclf1.predict_proba(x_test)

#with open('/home/rashmi/Dataset/single_file/xgb_classifier.pkl', 'rb') as file:  
 #   Pickled_Model = pickle.load(file)
    
#pred_probs=Pickled_Model.predict_proba(x_test)

print(pred_probs)

[[0.02598576 0.0575399  0.05910216 0.00634903 0.80682313 0.0442    ]
 [0.0934111  0.28716817 0.26363334 0.01930444 0.06782994 0.268653  ]
 [0.26227564 0.14029042 0.2963485  0.00386859 0.03140042 0.26581645]
 [0.02413877 0.0914949  0.07761557 0.6298246  0.02348149 0.15344477]
 [0.05900431 0.26729223 0.15410924 0.00298566 0.1915403  0.32506824]]


In [41]:
emo_array=[]
range1 = len(pred_probs)
for x in range(0,range1):
    index = pred_probs[x].argmax(axis=0).item()
    emo_array.append(emo_keys[index]);
    print(emo_keys[index]) 
print(len_wav_array)

sur
hap
sad
fea
neu
5


In [42]:
columns = ['wav_file','emotions','value']
df_features = pd.DataFrame(columns=columns)
for a in range(0,len_wav_array):
    try :
        feature_list =[wav_array[a]]
        feature_list.append(emo_array[a])
        #feature_list =[emo_array[a]]
        if emo_array[a] == "ang" :
            feature_list.append(0)
        elif emo_array[a] == "fea" :
            feature_list.append(1)
        elif emo_array[a] == "sad" :
            feature_list.append(2)
        elif emo_array[a] == "sur" :
            feature_list.append(3)
        elif emo_array[a] == "neu" :
            feature_list.append(4)
        elif emo_array[a] == "happy" :
            feature_list.append(5)
        else :
            feature_list.append(6)
        
        df_features = df_features.append(pd.DataFrame(feature_list, index=columns).transpose(), ignore_index=True)
    except :
        print("Error while making csv");

df_features.to_csv('/home/rashmi/Dataset/single_file/pre-processed-mfcc/before_final.csv', index=False)

In [43]:
#prioritize 
df= pd.read_csv('/home/rashmi/Dataset/single_file/pre-processed-mfcc/before_final.csv')
result = df.sort_values('value')
result

,wav_file,emotions,value
3,Ses05F_impro08_F007.wav,fea,1
2,Ses05f_impro08_F022.wav,sad,2
0,Ses05M_script02_2_F034.wav,sur,3
4,Ses05F_impro01_F013.wav,neu,4
1,ses05m_impro06_M004.wav,hap,6
